In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

# Shift the Overt labels to be zero-indexed
overt_labels_shifted = overt_labels - 11  # Subtract the minimum label (11)

# Shift the Covert labels to be zero-indexed
covert_labels_shifted = covert_labels - 21  # Subtract the minimum label (21)

# Now convert to categorical
overt_labels_categorical = to_categorical(overt_labels_shifted, num_classes=5)
covert_labels_categorical = to_categorical(covert_labels_shifted, num_classes=5)

# Train-test split for Overt data
X_overt_train, X_overt_test, y_overt_train, y_overt_test = train_test_split(overt_data, overt_labels_categorical, test_size=0.2, random_state=42)

# Train-test split for Covert data
X_covert_train, X_covert_test, y_covert_train, y_covert_test = train_test_split(covert_data, covert_labels_categorical, test_size=0.2, random_state=42)

# Transpose the input data so that it matches the expected shape
X_overt_train = X_overt_train.transpose((0, 2, 1))  # Shape becomes (batch_size, time_steps, features)
X_overt_test = X_overt_test.transpose((0, 2, 1))  # Same for test data
X_covert_train = X_covert_train.transpose((0, 2, 1))  # Shape becomes (batch_size, time_steps, features)
X_covert_test = X_covert_test.transpose((0, 2, 1))  # Same for test data

# Define the learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch < 5:  # Warm-up phase: increase the learning rate gradually
        return lr * (epoch + 1) / 5
    if epoch % 5 == 0 and epoch != 0:
        return lr * 0.9  # Reduce by 10% every 5 epochs
    return lr
# Define the Bi-LSTM model architecture with L2 regularization
def create_bi_lstm_model(input_shape):
    model = Sequential()

    # First Bidirectional LSTM Layer with L2 regularization
    model.add(Bidirectional(LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01)), input_shape=input_shape))  # L2 regularization
    model.add(Dropout(0.3))

    # Second Bidirectional LSTM Layer with L2 regularization
    model.add(Bidirectional(LSTM(64, return_sequences=False, kernel_regularizer=l2(0.01))))  # L2 regularization
    model.add(Dropout(0.3))

    # Fully connected layer with L2 regularization
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.01)))  # L2 regularization

    # Output layer with softmax activation for 5 classes
    model.add(Dense(5, activation='softmax'))  # 5 classes

    # Compile the model with RMSProp optimizer
    model.compile(optimizer=RMSprop(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Model for Overt Data
model_overt = create_bi_lstm_model((1000, 128))
model_overt.summary()

# Train the Bi-LSTM model for Overt Data with learning rate scheduler
history_overt = model_overt.fit(X_overt_train, y_overt_train, epochs=10, batch_size=32, validation_data=(X_overt_test, y_overt_test), callbacks=[LearningRateScheduler(lr_scheduler)])

# Evaluate the model on Overt data
test_loss_overt, test_accuracy_overt = model_overt.evaluate(X_overt_test, y_overt_test)
print(f"Overt Test Accuracy: {test_accuracy_overt * 100:.2f}%")

# Model for Covert Data
model_covert = create_bi_lstm_model((1000, 128))
model_covert.summary()

# Train the Bi-LSTM model for Covert Data with learning rate scheduler
history_covert = model_covert.fit(X_covert_train, y_covert_train, epochs=10, batch_size=32, validation_data=(X_covert_test, y_covert_test), callbacks=[LearningRateScheduler(lr_scheduler)])

# Evaluate the model on Covert data
test_loss_covert, test_accuracy_covert = model_covert.evaluate(X_covert_test, y_covert_test)
print(f"Covert Test Accuracy: {test_accuracy_covert * 100:.2f}%")

# Plot training and validation accuracy for Overt Data and Covert Data
plt.figure(figsize=(12, 6))

# Plot for Overt Data
plt.subplot(1, 2, 1)
plt.plot(history_overt.history['accuracy'], label='Train Accuracy')
plt.plot(history_overt.history['val_accuracy'], label='Test Accuracy')
plt.title('Overt Bi-LSTM Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot for Covert Data
plt.subplot(1, 2, 2)
plt.plot(history_covert.history['accuracy'], label='Train Accuracy')
plt.plot(history_covert.history['val_accuracy'], label='Test Accuracy')
plt.title('Covert Bi-LSTM Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()
